<a href="https://colab.research.google.com/github/Kennenzz/predict-employee-attrition/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**XÂY DỰNG MÔ HÌNH DỰ ĐOÁN NGUY CƠ NGHỈ VIỆC CỦA NHÂN VIÊN**
---

#Import các thư viện sẽ sử dụng trong đồ án bao gồm:
*   **Pandas**
*   **Numpy**
*   **Matplotlib**
*   **Seaborn**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


---
#Đọc file dataset có được từ trang web *Kaggle.com*

In [3]:
try:
    df = pd.read_csv('/content/WA_Fn-UseC_-HR-Employee-Attrition.csv')
    display(df.head())
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file dataset trong dữ liệu folder")
except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Lỗi: Không tìm thấy file dataset trong dữ liệu folder


---
#Mô tả tổng quan về dataset

In [4]:
display(df.info())
display(df.describe())
display(df.shape)

NameError: name 'df' is not defined

#Tổng quan:
Dataset gồm 35 cột và 1470 dòng dữ liệu.

---
# Phân loại các biến trong dataset

In [ ]:
# cần run file từ đầu đến cuối
numerical_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

print("Biến số:")
display(numerical_cols)

print("\nBiến phân loại:")
display(categorical_cols)

---
#**Tiền xử lý dữ liệu**
1.   Xử lý dữ liệu thiếu

In [ ]:
df.isnull().sum()


**Không có dữ liệu thiếu trong dataset**

2.   Xử lý dữ liệu dư thừa, dữ liệu nhiễu (không liên quan)

In [ ]:
df.drop(columns=['Over18', 'EmployeeNumber', 'EmployeeCount'], axis =1, inplace= True)

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

Qua quan sát dữ liệu thì có thể thấy các cột dữ liệu không cần thiết như sau:
*   **Over18** tất cả nhân viên đều đã trên 18 tuổi
*   **EmployeeCount** luôn có giá trị là 1
*   **EmployeeNumber** luôn có giá trị khác nhau như việc nhân viên có mã nhân viên


3.   Xử lý mã hóa biến phân loại

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable 'Attrition'
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

# Identify categorical columns (excluding 'Attrition' as it's already encoded)
categorical_cols = df.select_dtypes(include='object').columns

# Apply encoding to the remaining categorical columns
for col in categorical_cols:
    if df[col].nunique() == 2:
        # Apply Label Encoding for binary columns
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    else:
        # Apply One-Hot Encoding for columns with more than two unique values
        df = pd.get_dummies(df, columns=[col], drop_first=True)

display(df.head())

---
#**KHAI PHÁ DỮ LIỆU EDA**


In [ ]:
# 1. Count the number of employees who stayed (0) and left (1)
attrition_counts = df['Attrition'].value_counts()
print("Số lượng nhân viên nghỉ việc và không nghỉ việc:")
display(attrition_counts)

# 2. Visualize the distribution using a countplot
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Attrition', palette='viridis')
plt.title('Tỷ lệ nghỉ việc vs ở lại')
plt.xticks([0, 1], ['Ở lại', 'Nghỉ việc'])
plt.xlabel('Nghỉ việc')
plt.ylabel('Số lượng nhân viên')
plt.show()

# 3. Display the percentage of attrition
attrition_percentage = (attrition_counts[1] / len(df)) * 100
print(f"\nPhần trăm nhân viên nghỉ việc: {attrition_percentage:.2f}%")